# CC6204 Deep Learning, Universidad de Chile
## Código de Red Neuronal "casi a mano", usando el `torch.autograd` 

El código está pensado para acompañar la clase de Back Propagation de CC6204 y servir como una introducción rápida a las funcionalidades básicas de pytorch, incluyendo autograd. No está pensado en ser un código modular, si no más bien un código pedagógico para los temas de grafo de computación, back propagation, y descenso de gradiente por paquetes. Es una versión usando autograd del código que aparece [acá](https://colab.research.google.com/drive/1LZ21iEeIX97kb_XEgv7bw-DQyITID_zm).

(Pensado para correr en [Colaboratory](http://colab.research.google.com))



In [0]:
# http://pytorch.org/
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.0.post4-{platform}-linux_x86_64.whl torchvision
!pip install -q ipdb

In [0]:
import torch
import numpy as np
import sys
import time
import ipdb

# Genera una semilla fija para que los experimentos sea repetibles.
t_cg = torch.manual_seed(1547)

In [0]:
# Chequeamos si hay acceso a la GPU.
print(torch.cuda.is_available())

In [0]:
# Funciones de activación:

# Función sigmoid, recibe un objeto torch.Tensor
def sig(T):
  return torch.reciprocal(1 + torch.exp(-1 * T))

# Función tanh, recibe un objeto torch.Tensor
def tanh(T):
  E = torch.exp(T)
  e = torch.exp(-1 * T)
  return (E - e) * torch.reciprocal(E + e)

# Función de pérdida
def bi_cross_ent_loss(y_pred, y, safe=True, epsilon=1e-5):
  N = y.size()[0]
  
  if safe:
    # Asegura que no haya valores indefinidos.
    y_pred.data.clamp_(epsilon, 1-epsilon)
  
  B = (1-y) * torch.log(1-y_pred) + y * torch.log(y_pred)
  return -1/N * torch.sum(B)  

In [0]:
# Para elegir el siguiente batch (uno al azar) desde los datos de entrada
def elige_batch(X,Y,b):
  N = X.size()[0]
  x_lista = []
  y_lista = []
  
#  i = np.random.randint(N-b) # <-- descomentar esto para ejemplos
  for _ in range(b):
    i = np.random.randint(N) # <-- comentar esto para ejemplos
    x_lista.append(X[i:i+1])
    y_lista.append(Y[i:i+1])
  
  x = torch.cat(x_lista, dim=0)
  y = torch.cat(y_lista, dim=0)
  
  return x,y

In [0]:
def ejemplo_FFNN(X, Y, b=1, d1=200, d2=300, lr=0.06, 
                 epochs=10, run_in_GPU=True, reports_every=1, 
                 cheq_grad=False, init_v=1):
  
  # Define un tipo para los tensores según si correrá en la GPU o no
  if run_in_GPU:
    assert torch.cuda.is_available()
    t_type = torch.cuda.FloatTensor
  else:
    t_type = torch.FloatTensor
    
  # Numero de ejemplos y cantidad de features
  N = X.size()[0]
  f = X.size()[1]
  
  # d0 es la cantidad de features  
  d0 = f
  
  # Crea los tensores de parámetros
  W1 = torch.randn(d0,d1).type(t_type) * init_v
  b1 = torch.zeros(d1).type(t_type)
  W2 = torch.randn(d1,d2).type(t_type) * init_v
  b2 = torch.zeros(d2).type(t_type)
  U = torch.randn(d2,1).type(t_type) * init_v
  c = torch.randn(1).type(t_type) 
  
  ###############################
  # Convierte todo en variables
  W1 = torch.autograd.Variable(W1, requires_grad=True)
  b1 = torch.autograd.Variable(b1, requires_grad=True)
  W2 = torch.autograd.Variable(W2, requires_grad=True)
  b2 = torch.autograd.Variable(b2, requires_grad=True)
  U = torch.autograd.Variable(U, requires_grad=True)
  c = torch.autograd.Variable(c, requires_grad=True)
  ###############################
  
  parametros = {'W1':W1, 'b1':b1, 'W2':W2, 'b2':b2, 'U':U, 'c':c}
  
  # Cuenta los parámetros en total
  cant_parametros = 0
  for P in parametros:
    cant_parametros += parametros[P].nelement()      
  print('Cantidad de parámetros:', cant_parametros)
     
  tiempo_epochs = 0
  for e in range(1,epochs+1):  
    inicio_epoch = time.clock()
 
    # Cantidad de iteraciones por epoch (b es el tamaño del batch)
    I = int(N/b) 
    
    for i in range(I):
      x, y = elige_batch(X,Y,b)
      # Asegura de pasarlos a la GPU si fuera necesario
      x = x.type(t_type)
      y = y.type(t_type)
      
      ###############################
      # Convierte en variables
      x = torch.autograd.Variable(x)
      y = torch.autograd.Variable(y)
      ###############################
      
      # Computa la pasada hacia adelante (forward)
      # No es necesario explicitar los calculos intermedios
      h1 = tanh(x.mm(W1).add(b1))
      h2 = sig(h1.mm(W2).add(b2))
      y_pred = sig(h2.mm(U).add(c))
                  
      # Computa la función de pérdida
      L = bi_cross_ent_loss(y_pred,y) 
      
      ###############################
      # Todas las derivadas las puede
      # computar autograd
      L.backward()
      # Esta instrucción ejecuta el 
      # backpropagation desde L
      ###############################
      
      # Actualiza los pesos
      for P in parametros:
        parametros[P].data -= lr * parametros[P].grad.data
        parametros[P].grad.data.zero_() 
        # esto último lo necesitamos porque los gradientes
        # se acumulan en el backpropagation (ver clase de
        # backpropagation e implementación iterativa).
      
    tiempo_epochs += time.clock() - inicio_epoch
    
    if e % reports_every == 0:
      # Calcula la certeza de las predicciones sobre todo el conjunto
      X = X.type(t_type) # pasa a la GPU si fuera necesario
      Y = Y.type(t_type) # pasa a la GPU si fuera necesario

      # Predice usando la red
      H1 = tanh(X.mm(W1.data).add(b1.data))
      H2 = sig(H1.mm(W2.data).add(b2.data))
      Y_PRED = sig(H2.mm(U.data).add(c.data))
      
      # Calcula la pérdida de todo el conjunto
      L_total = bi_cross_ent_loss(Y_PRED, Y, safe=False)

      # Elige una clase dependiendo del valor de Y_PRED
      Y_PRED_BIN = (Y_PRED >= 0.5).float()

      correctos = torch.sum(Y_PRED_BIN == Y)
      acc = correctos / N * 100

      sys.stdout.write(
            '\rEpoch:{0:03d}'.format(e) + ' Acc:{0:.2f}%'.format(acc)
            + ' Loss:{0:.4f}'.format(L_total) 
            + ' Tiempo/epoch:{0:.3f}s'.format(tiempo_epochs/e)) 
  
  return parametros

In [0]:
N = 5000 # numero de ejemplos
f = 300 # numero de features


X = torch.rand(N,f)
X = torch.bernoulli(X)

Y = torch.rand(N,1)
Y = torch.bernoulli(Y)

In [0]:
epochs = 30

red = ejemplo_FFNN(X, Y, b=32, d1=300, d2=200, epochs=epochs, 
             run_in_GPU=True, lr=0.06, init_v=0.8)

Para ejemplificar, probar el anterior código de la siguiente forma:

*   Cambiar el tamaño del batch desde 2 a 1000: visualizar tiempo de entrenamiento vs Acc
*   Cambiar el tamaño de las capas / número de parámetros: visualizar tiempo de entrenamiento
*   Cambiar el valor máximo de inicialización (0.01,1,1.5,2)
*   Mostrar que se pueden calcular (aun) más eficiente la pasada hacia atrás reutilizando algunos valores previamente computados cuando derivamos `sig` y `tanh`: visualizar el tiempo de entrenamiento.
*   Descomentar la línea del generador de batches. ¿Cómo se explica el resultado?


Código por Jorge Pérez

https://github.com/jorgeperezrojas

@perez